In [3]:
import os

In [4]:
%pwd

'e:\\Deep-learning-credit-card-fraud-detection\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'e:\\Deep-learning-credit-card-fraud-detection'

In [8]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir:Path
    processor_dir :Path
    train_dir :Path
    test_dir :Path

    

In [9]:
from src.card_fraud_detection.utilis.commmon import create_directories,save_model
from src.card_fraud_detection.utilis.commmon import read_yaml
from src.card_fraud_detection.constants import config_file_path,params_file_path

In [11]:
class Confuguration:
    def __init__(self,confi_path=config_file_path,params_path=params_file_path):
        self.config=read_yaml(confi_path)
        self.params=read_yaml(params_path)

        create_directories([self.config.Artifacts])

    def data_transformation_config(self)->DataTransformationConfig:
        transformation=self.config.data_transformation
        ingestion=self.config.Data_ingestion

        create_directories([transformation.root_dir])

        data_transformation=DataTransformationConfig(
            root_dir=transformation.root_dir,
            processor_dir=transformation.processor_dir,
            train_dir=ingestion.train_dir,
            test_dir=ingestion.test_dir

        )
        return data_transformation

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import pandas as pd

In [18]:
class DataTransformation:
    def __init__(self,config):
        self.config = config

    def get_the_data_transformation(self):

        cat_features =['city','state']
        numerical_features= ['transaction_dollar_amount','Long','Lat','credit_card_limit']

        num_pip=Pipeline(steps=[
            ('impueter',SimpleImputer(strategy='mean')),
            ('scaler',StandardScaler())])
        cat_pip=Pipeline(steps=[
            ('impueter',SimpleImputer(strategy='most_frequent')),
            ('coder',OneHotEncoder())
            ])
        preprocessor=ColumnTransformer([
            ('num_pip',num_pip,numerical_features),
            ('cat_pip',cat_pip,cat_features)])
        
        return preprocessor
    
    def data_transformation_initiated(self,):

     

        train_df=pd.read_csv(self.config.train_dir)

        test_df=pd.read_csv(self.config.test_dir)

        train_df.dropna()  #dropping the null values

        test_df.dropna() #dropping the null values

        train_df.drop(['zipcode','date','credit_card'],axis=1,inplace=True)

        test_df.drop(['zipcode','date','credit_card'],axis=1,inplace=True)

        target_feature=['Is_fraud']

        x_train,y_train,x_test,y_test = (train_df.iloc[:,:-1],train_df[target_feature],test_df.iloc[:,:-1],test_df[target_feature])
        
        print(x_train.shape)
        print(y_train.shape)
        preprocessor= self.get_the_data_transformation()


        x_train=preprocessor.fit_transform(x_train)

        x_test= preprocessor.transform(x_test)



        x_train=x_train.toarray() #convert to array from sparse matrix

        x_test= x_test.toarray() #convert to array from sparse matrix

        save_model(filepath=self.config.processor_dir,obj=preprocessor)

        return(x_train,y_train,x_test,y_test)

In [20]:

try:
    data_transformation=Confuguration()

    config=data_transformation.data_transformation_config()

    data_transformation=DataTransformation(config)

    x_train,y_train,x_test,y_test=data_transformation.data_transformation_initiated()

    
except Exception as e:
    raise e


[2023-08-30 15:06:51,812: INFO: commmon: file created]
[2023-08-30 15:06:51,812: INFO: commmon: file created]
(235670, 6)
(235670, 1)
(235670, 163)
(58918, 163)
(58918, 1)
(235670, 1)
